# Fifth model 

This model will try to predict exam level labels for positive patients

## Imports

In [1]:
import numpy as np
from balance import *
from model import *
from torchsummary import summary

In [2]:
import torch
torch.cuda.empty_cache()

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

PyTorch version: 1.7.1
GPU found :)


# Load datasets

In [3]:
dataset = torch.load('datasets/multi_image_multi_label_dataset.pt')
nb_images = 50

## CNN

In [4]:
class ModelV5(nn.Module):
    
    def __init__(self, input_channels=3*nb_images):
        """Convnet with 4 convolution layer + pooling + BN, with 3 fully connected at the end"""
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=256, kernel_size=5)
        self.bn1 = nn.BatchNorm2d(256)
        self.conv2 = nn.Conv2d(256, 512, 3)
        self.bn2 = nn.BatchNorm2d(512)
        self.conv3 = nn.Conv2d(512, 256, 3)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 64, 3)
        self.bn4 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(64*14*14 , 512)
        self.fc2 = nn.Linear(512, 128)
        
        self.fc_location = nn.Linear(128, 3)
        self.fc_ratio = nn.Linear(128, 2)
        self.fc_type = nn.Linear(128, 3)
        
        self.dropout = nn.Dropout(0.5)
        
        self.softmax = nn.LogSoftmax(dim=1)
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 64*14*14) # Flatten
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        
        location_out = self.sigmoid(self.fc_location(x))
        ratio_out = self.softmax(self.fc_ratio(x))
        type_out = self.softmax(self.fc_type(x))
        
        return location_out, ratio_out, type_out

In [5]:
model = ModelV5().to(device)
#summary(model, (150, 256, 256))

## Split data into train and test

In [6]:
batch_size = 1
ratio = 0.2

train_loader, test_loader = train_test_split(dataset, ratio, batch_size)

print(f'Train size: {len(train_loader)}')
print(f'Test size: {len(test_loader)}')

Train size: 72
Test size: 18


## Model Training

In [ ]:
num_epoch = 30
step_count = len(train_loader)
MSE = nn.MSELoss()
NLL = nn.NLLLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001)

losses = list()
losses_ratio = list()
losses_location = list()
losses_type = list()

save_filename = './models/model_v5.h5'

for epoch in range(num_epoch):
    
    epoch_loss = 0
    epoch_loss_ratio = 0
    epoch_loss_location = 0
    epoch_loss_type = 0
    
    for i, sample in enumerate(train_loader):
        
        images = sample['image'].to(device, dtype=torch.float)
        ratios = sample['ratios']
        locations = sample['locations']
        types = sample['types']
        
        optimizer.zero_grad()
        
        location_out, ratio_out, type_out = model(images)
        
        # Compute loss for each category of labels
        loss_ratio = NLL(ratio_out, ratios.to(device))
        loss_location = MSE(location_out, locations.to(device))
        loss_type = NLL(type_out, types.to(device))
        
        # Keep track of separate losses
        epoch_loss_ratio += loss_ratio
        epoch_loss_location += loss_location
        epoch_loss_type += loss_type

        loss = loss_ratio + loss_location + loss_type
        epoch_loss += loss
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        
        if ((i+1) % int(step_count/10)) == 0:
            print(f"Epoch [{epoch + 1}/{num_epoch}]" f", step [{i + 1}/{step_count}]" f", loss: {loss.item():.4f}")
    
    losses.append(epoch_loss)
    losses_ratio.append(epoch_loss_ratio)
    losses_location.append(epoch_loss_location)
    losses_type.append(epoch_loss_type)

In [ ]:
torch.save(model.state_dict(), './models./model_v5')

In [ ]:
plot_loss(losses, num_epoch, len(train_loader), 'Global loss model v5 study level.png')

## Test the model

In [ ]:
# Load model for evaluation
trained_model = ModelV5()
trained_model.load_state_dict(torch.load('./models/model_v5.h5'))
trained_model.to(device)